In [1]:
print('hellow world')

hellow world


In [26]:
import numpy as np

In [3]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.ensemble import (
    RandomForestClassifier, 
    ExtraTreesClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
)

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from xgboost import XGBClassifier


In [4]:
df = pd.read_csv("data/bigboy.csv")

In [5]:
x = df[['cosine_feature', 'count_feature', 'hr_feature', 'time_feature']]
y = df['psg_label']

In [6]:
x, y

(       cosine_feature  count_feature  hr_feature  time_feature
 0           -0.258819       0.860988    0.278078      0.000000
 1           -0.260926       0.618550    0.270274      0.008333
 2           -0.263031       0.608050    0.260905      0.016667
 3           -0.265135       0.891078    0.252579      0.025000
 4           -0.267238       1.552097    0.244514      0.033333
 ...               ...            ...         ...           ...
 25476       -0.751840       0.000000    0.130491      7.750000
 25477       -0.750400       0.000000    0.134184      7.758333
 25478       -0.748956       0.000000    0.138695      7.766667
 25479       -0.747508       0.000000    0.142484      7.775000
 25480       -0.746057       0.000000    0.148858      7.783333
 
 [25481 rows x 4 columns],
 0        0
 1        0
 2        0
 3        0
 4        0
         ..
 25476    5
 25477    5
 25478    5
 25479    5
 25480    5
 Name: psg_label, Length: 25481, dtype: int64)

In [7]:
# List discrete values in y

y.unique()

array([0, 1, 2, 3, 5])

In [8]:
# Count occurrences of each value in y

y.value_counts()

psg_label
2    12483
5     5540
3     3545
0     2152
1     1761
Name: count, dtype: int64

In [9]:
y.replace({5: 4}, inplace=True)

In [10]:
y.unique()

array([0, 1, 2, 3, 4])

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [12]:
x_train.shape, y_train.shape

((20384, 4), (20384,))

In [13]:
x_test.shape, y_test.shape

((5097, 4), (5097,))

In [14]:
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [15]:
y_pred = clf.predict(x_test)

In [16]:
clf.feature_importances_

array([0.21239651, 0.15867127, 0.38317714, 0.24575508])

In [27]:
adjacency_map = {
    0: {0, 1, 5},        # Wake (0) is adjacent to N1 (1) and REM (5)
    1: {1, 0, 2},        # N1 (1) is adjacent to Wake (0) and N2 (2)
    2: {2, 1, 3, 5},     # N2 (2) is adjacent to N1 (1), N3 (3), and REM (5)
    3: {3, 2},           # N3 (3) is adjacent to N2 (2)
    5: {5, 2, 0}         # REM (5) is adjacent to N2 (2) and Wake (0)
}

def get_adjacent_accuracy(y_true, y_pred):
    # Convert to list/array if they are pandas Series
    y_true = np.array(list(y_true))
    y_pred = np.array(list(y_pred))
    
    correct_count = 0
    total_count = len(y_true)
    
    for t, p in zip(y_true, y_pred):
        if p in adjacency_map.get(t, {t}):
            correct_count += 1
            
    return correct_count / total_count

In [28]:
import os
import joblib

# Create a directory to save the models
models_dir = "saved_models"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

algorithms = {
    'Random Forest': RandomForestClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000),
    'XGBoost': XGBClassifier()
}

for name, clf in algorithms.items():
    clf.fit(x_train, y_train)

    y_pred = clf.predict(x_test)
    exact_score = clf.score(x_test, y_test)
    adj_score = get_adjacent_accuracy(y_test, y_pred)
    print(f"{name:<20} | {exact_score:.4f}     | {adj_score:.4f}")

    # Save the model
    model_filename = os.path.join(models_dir, f"{name.replace(' ', '_')}.joblib")
    joblib.dump(clf, model_filename)
    print(f"Saved {name} to {model_filename}")
    # print(f"{name}: {clf.features_}")
    # print(f"{name}: {clf.feature_importances_}")


Random Forest        | 0.6775     | 0.8140
Saved Random Forest to saved_models/Random_Forest.joblib
Extra Trees          | 0.6349     | 0.7928
Saved Extra Trees to saved_models/Extra_Trees.joblib
Extra Trees          | 0.6349     | 0.7928
Saved Extra Trees to saved_models/Extra_Trees.joblib
AdaBoost             | 0.5021     | 0.6798
Saved AdaBoost to saved_models/AdaBoost.joblib
AdaBoost             | 0.5021     | 0.6798
Saved AdaBoost to saved_models/AdaBoost.joblib
Gradient Boosting    | 0.6070     | 0.7665
Saved Gradient Boosting to saved_models/Gradient_Boosting.joblib
Logistic Regression  | 0.5221     | 0.7145
Saved Logistic Regression to saved_models/Logistic_Regression.joblib
K-Nearest Neighbors  | 0.6088     | 0.7706
Saved K-Nearest Neighbors to saved_models/K-Nearest_Neighbors.joblib
Gradient Boosting    | 0.6070     | 0.7665
Saved Gradient Boosting to saved_models/Gradient_Boosting.joblib
Logistic Regression  | 0.5221     | 0.7145
Saved Logistic Regression to saved_models/Log

/Users/kainishikawa/Desktop/sleep_classifiers/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Neural Network       | 0.5900     | 0.7400
Saved Neural Network to saved_models/Neural_Network.joblib
XGBoost              | 0.6482     | 0.7977
Saved XGBoost to saved_models/XGBoost.joblib
XGBoost              | 0.6482     | 0.7977
Saved XGBoost to saved_models/XGBoost.joblib


In [47]:
import os
import joblib

# Create a directory to save the models
models_dir = "saved_models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

In [48]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

In [52]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Save the scaler
scaler_filename = os.path.join(models_dir, "scaler.joblib")
joblib.dump(scaler, scaler_filename)
print(f"Saved scaler to {scaler_filename}")

model = Sequential([
    Dense(64, activation='relu', input_shape=(4,)),  
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(y.unique()), activation='softmax')  
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(x_train_scaled, y_train, 
                    epochs=100, 
                    batch_size=32, 
                    validation_split=0.2,
                    verbose=0)

# Evaluate
test_loss, test_accuracy = model.evaluate(x_test_scaled, y_test, verbose=0)
print(f"Neural Network (TensorFlow): {test_accuracy:.4f}")

# Save the TensorFlow model
tf_model_filename = os.path.join(models_dir, "tensorflow_model.h5")
model.save(tf_model_filename)
print(f"Saved TensorFlow model to {tf_model_filename}")

Saved scaler to saved_models/scaler.joblib


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Neural Network (TensorFlow): 0.5764
Saved TensorFlow model to saved_models/tensorflow_model.h5


In [50]:
import os
import joblib

# Create a directory to save the models
models_dir = "saved_models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

In [1]:
# Example of loading a saved model
import joblib

# Load the Random Forest model
loaded_rf_model = joblib.load(os.path.join(models_dir, "Random_Forest.joblib"))

# You can now use the loaded model to make predictions
# For example, on the test set
predictions = loaded_rf_model.predict(x_test)
print("Predictions from loaded Random Forest model:")
print(predictions)

# # Example of loading the TensorFlow model
# from tensorflow.keras.models import load_model

# loaded_tf_model = load_model(os.path.join(models_dir, "tensorflow_model.h5"))

# # You can now use the loaded model to make predictions
# # Make sure to scale the input data just like you did for training
# predictions_tf = loaded_tf_model.predict(x_test_scaled)
# print("Predictions from loaded TensorFlow model:")
# print(predictions_tf)

NameError: name 'os' is not defined